In [1]:
import ROOT
import root_numpy as rnpy
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn
import pickle
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM

Welcome to ROOTaaS 6.06/08


Using Theano backend.


In [2]:
# build here the keras model
def RNN_classifier():
    model = Sequential()
    
    model.add(LSTM(32, input_shape = (None, 8)))
    
    # make an output layer with just 1 output -> for a binary classification problem: b-jet / not b-jet
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  
    return model

In [9]:
def prepare_training_data(jet_list, label):
    # extract the tracks and put them in pt-order, hardest tracks first
    jet_tracks = [cur[-1] for cur in jet_list]
    jet_tracks = [sorted(cur, key = lambda tracks: tracks[0], reverse = True) for cur in jet_tracks]
    
    # zero-pad the track dimension, to make sure all jets fed into the network during training have the same length
    max_tracks = max([len(cur) for cur in jet_tracks])
    padded = [np.vstack([cur, np.full((max_tracks - len(cur), 8), 0, float)]) for cur in jet_tracks]
    
    batch_size = len(padded)
    timestep_size = max_tracks
    jet_dim = 8
    x_train = np.array(padded).reshape(batch_size, timestep_size, jet_dim)
    y_train = np.full((batch_size, 1), 1, float) # all are b-jets!
    
    return x_train, y_train, batch_size

In [4]:
batch_size_jets = 10
batch_size_tracks = 50
read_pos_jets = 0
read_pos_tracks = 0
number_chunks = 0
chunks_limit = 1

In [5]:
while number_chunks < chunks_limit:
    number_chunks += 1
    
    # read in new chunk of jet and track data
    d1 = pd.DataFrame(rnpy.root2array("/mnt/t3nfs01/data01/shome/jpata/btv/gc/TagVarExtractor/GCa08e5e237323/TT_TuneCUETP8M1_13TeV-powheg-pythia8/job_0_out.root",
                                 treename = "tagVars/ttree", start = read_pos_jets, stop = read_pos_jets + batch_size_jets))
    d2 = pd.DataFrame(rnpy.root2array("/mnt/t3nfs01/data01/shome/jpata/btv/gc/TagVarExtractor/GCa08e5e237323/TT_TuneCUETP8M1_13TeV-powheg-pythia8/job_0_out.root",
                                 treename = "tagVars/ttree_track", start = read_pos_tracks, stop = read_pos_tracks + batch_size_tracks))
    
    # figure out where the next chunk should start so that we don't count any jets multiple times
    last_tracks = (int)(d2.tail(1)['Track_jetIndex'].iloc[0]-1)
    last_jet = (int)(d1.tail(1)['Jet_jetIndex'].iloc[0]-1)
    read_pos_jets += (d1.loc[d1['Jet_jetIndex'] == last_tracks].index[-1] + 1)
    read_pos_tracks += (d2.loc[d2['Track_jetIndex'] == last_tracks].index[-1] + 1)

    # add the track data to the jet list
    d1['track_data'] = pd.np.empty((len(d1.index),0)).tolist()
    
    # iterate over the track list to join jets with the tracks belonging to them
    for irow, row in d2.iterrows():
        # these are the track data of the current track:
        tracks = row[["Track_pt", "Track_eta", "Track_phi", "Track_dxy", "Track_dz", "Track_IP", "Track_IP2D", "Track_length"]].as_matrix()
        jet_index = int(row["Track_jetIndex"])
        if jet_index > last_tracks:
            break
        table_index = d1.loc[d1['Jet_jetIndex'] == jet_index].index[0]

        # append the tracks data to the matching jet in the main table
        d1['track_data'][table_index].append(tracks)
    
    # now divide the jets and put them in separate lists, according to their flavour
    jets_b = []
    jets_l = []
    jets_c = []

    # iterate over the jet list, with already matched tracks
    for irow, row in d1.iterrows():
        jet_index = int(row["Jet_jetIndex"])
        if jet_index > last_tracks:
            break

        flavour = int(row["Jet_flavour"])

        # select the right list this jet belongs to
        if abs(flavour) == 5:
            jets = jets_b
        elif abs(flavour) == 4:
            jets = jets_c
        else:
            jets = jets_l

        # add the new jet to the list
        jets += [(row["Jet_pt"], row["Jet_eta"], row["Jet_phi"], row["Jet_mass"], flavour, row["track_data"])]
        
    # now, have sorted jets in three lists, can use them directly for training!

/swshare/anaconda/lib/python2.7/site-packages/root_numpy/_tree.py:209: RuntimeWarning: ignoring duplicate branch named 'Track_nHitPixel'
  warn_missing_tree)


In [10]:
x_train, y_train, batch_size = prepare_training_data(jets_b, 1)

In [11]:
model = RNN_classifier()

In [12]:
model.fit(x_train,y_train, batch_size = batch_size)

Epoch 1/10
3/3 [==============================] - 0s - loss: 0.8104 - acc: 0.0000e+00
Epoch 2/10
3/3 [==============================] - 0s - loss: 0.7773 - acc: 0.3333
Epoch 3/10
3/3 [==============================] - 0s - loss: 0.7451 - acc: 0.6667
Epoch 4/10
3/3 [==============================] - 0s - loss: 0.7137 - acc: 0.6667
Epoch 5/10
3/3 [==============================] - 0s - loss: 0.6832 - acc: 0.6667
Epoch 6/10
3/3 [==============================] - 0s - loss: 0.6536 - acc: 0.6667
Epoch 7/10
3/3 [==============================] - 0s - loss: 0.6250 - acc: 0.6667
Epoch 8/10
3/3 [==============================] - 0s - loss: 0.5973 - acc: 0.6667
Epoch 9/10
3/3 [==============================] - 0s - loss: 0.5706 - acc: 0.6667
Epoch 10/10
3/3 [==============================] - 0s - loss: 0.5449 - acc: 1.0000


In [ ]:
# save the model after training here


In [64]:
model.predict(b, batch_size = 1)

INFO (theano.gof.compilelock): Refreshing lock /mnt/t3nfs01/data01/shome/phwindis/.theano/compiledir_Linux-2.6-el6.x86_64-x86_64-with-redhat-6.6-Carbon-x86_64-2.7.12-64/lock_dir/lock


array([[ 0.37535214]], dtype=float32)